In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)\
import missingno as msno
import matplotlib.pyplot as plt
import datetime as dt

In [2]:
merged = pd.read_csv('data/merged_state_data.csv')
merged.head(5)

,State,mean_elevation,Land Area (sq mi),Water Area (sq mi),Total Area (sq mi),Percentage of Federal Land,Urbanization Rate (%)
0,AK,580,570641,94743,665384,60.9%,64.9
1,AL,150,50645,1775,52420,2.7%,57.7
2,AR,200,52035,1143,53179,9.4%,55.5
3,AZ,1250,113594,396,113990,38.6%,89.3
4,CA,880,155779,7916,163695,45.4%,94.2


# Wethaer monthly state agreggate prep

In [3]:
weather_monthly = pd.read_csv('data/weather_monthly_state_aggregates.csv')
weather_monthly.head(5)

,State,year_month,PRCP,EVAP,TMIN,TMAX
0,AK,1992-05,5.36,44.17,-56.0,233.0
1,AK,1992-06,14.03,39.27,-17.0,322.0
2,AK,1992-07,16.78,30.91,28.0,300.0
3,AK,1992-08,10.46,20.14,-33.0,267.0
4,AK,1992-09,17.25,15.27,-61.0,183.0


In [10]:
weather_monthly['month'] = pd.to_datetime(weather_monthly['year_month']).dt.month
weather_monthly['year'] = pd.to_datetime(weather_monthly['year_month']).dt.year
weather_monthly.drop(columns=['year_month'], inplace=True)
weather_monthly.head(5)

,State,PRCP,EVAP,TMIN,TMAX,month,year
0,AK,5.36,44.17,-56.0,233.0,5,1992
1,AK,14.03,39.27,-17.0,322.0,6,1992
2,AK,16.78,30.91,28.0,300.0,7,1992
3,AK,10.46,20.14,-33.0,267.0,8,1992
4,AK,17.25,15.27,-61.0,183.0,9,1992


In [18]:
weather_monthly.shape

(11775, 7)

# WILDFIRE Dataset prep

In [7]:
wildfire_size = pd.read_csv('data/wildfire_sizes_before_2010.csv')

wildfire_size.rename(columns={'month': 'year_month'}, inplace=True)
wildfire_size.head(5)

,STATE,year_month,total_fire_size
0,AK,1992-04,10.1
1,AK,1992-05,4309.5
2,AK,1992-06,86460.6
3,AK,1992-07,48578.5
4,AK,1992-08,3321.8


In [9]:
wildfire_size['month'] = pd.to_datetime(wildfire_size['year_month']).dt.month
wildfire_size['year'] = pd.to_datetime(wildfire_size['year_month']).dt.year
wildfire_size.drop(columns=['year_month'], inplace=True)
wildfire_size.head(5)

,STATE,total_fire_size,month,year
0,AK,10.1,4,1992
1,AK,4309.5,5,1992
2,AK,86460.6,6,1992
3,AK,48578.5,7,1992
4,AK,3321.8,8,1992


In [17]:
wildfire_size.shape

(9632, 4)

# Concatenation

In [20]:
# Merge weather_monthly and wildfire_size on State, month, and year
merged_weather_wildfire = pd.merge(weather_monthly, wildfire_size, left_on=['State', 'month', 'year'], right_on=['STATE', 'month', 'year'])

# Drop the STATE column
merged_weather_wildfire.drop(columns=['STATE'], inplace=True)

# Merge the result with the merged dataframe on State
final_merged = pd.merge(merged_weather_wildfire, merged, left_on='State', right_on='State')

# Reorder columns to have 'month' and 'year' at iloc 1 and 2
cols = ['State', 'year', 'month'] + [col for col in final_merged.columns if col not in ['State', 'month', 'year', 'total_fire_size']] + ['total_fire_size']
final_merged = final_merged[cols]

final_merged.head()

,State,year,month,PRCP,EVAP,TMIN,TMAX,mean_elevation,Land Area (sq mi),Water Area (sq mi),Total Area (sq mi),Percentage of Federal Land,Urbanization Rate (%),total_fire_size
0,AK,1992,5,5.36,44.17,-56.0,233.0,580,570641,94743,665384,60.9%,64.9,4309.5
1,AK,1992,6,14.03,39.27,-17.0,322.0,580,570641,94743,665384,60.9%,64.9,86460.6
2,AK,1992,7,16.78,30.91,28.0,300.0,580,570641,94743,665384,60.9%,64.9,48578.5
3,AK,1992,8,10.46,20.14,-33.0,267.0,580,570641,94743,665384,60.9%,64.9,3321.8
4,AK,1992,9,17.25,15.27,-61.0,183.0,580,570641,94743,665384,60.9%,64.9,35.8


In [21]:
final_merged.shape

(6583, 14)

In [22]:
final_merged.to_csv('data/final_merged.csv', index=False)